In [ ]:

# Source the package setup script
source("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/scripts/00_setup_packages.R")

# Source the graphing functions
source("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/scripts/01_graphing_functions.R", local = knitr:::knit_global())


# Model 3

## Question

How do environmental factors (**microhabitat**, **nudibranch presence**) affect P. cristatus occurrence across individuals, sexes, and morphs?

## Objective

Test for the effect of **microhabitat type** and **aeolid nudibranch presence** on the occurrence of **morphs**. 

## Method

### 1. Load cleaned data.

We start by loading the cleaned data from the "02_data_cleaning" pipeline. This data has already undergone transformations and contains relevant metrics for our models.


In [ ]:

data_m3_clean <- read.csv("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/cleaned/data_m3_clean.csv")



### 2. Prepare data for modeling.

#### **Categorical predictors**

For categorical and binary predictors, R automatically dummy-codes the variables. For binary variables, such as Sex (0 = Female, 1 = Male), R sets the baseline to 0 unless specified otherwise. This ensures comparisons are made relative to the baseline group.

#### **Continuous predictors**

We standardize continuous predictors, such as Fecundity, by centering and scaling them (dividing by two standard deviations). This improves interpretability, aligning their coefficients with those of binary predictors, as suggested by Gelman (2008).

In [ ]:

# Convert categorical variables to factors
columns_to_convert_m3 <- c("Site", "Quadrat", "Microhabitat", "Nudibranch", "Red_Beige", "White", "Bright")

data_m3_clean <- data_m3_clean %>%
    mutate(across(all_of(columns_to_convert_m3), as.factor))

# Set reference category for categorical predictors
data_m3_clean$Microhabitat <- relevel(data_m3_clean$Microhabitat, ref = "Hydroid")


# Convert continuous/binary variables to numerical, if not already done so
# NONE ARE CONTINUOUS



---

### 3. Visualize response variable distributions

To understand the distributions of our response variables, we plot density curves. This helps confirm whether a Gamma distribution is appropriate for modeling, as Gamma is suited for positively skewed, continuous data.


In [ ]:
# Function to generate stacked bar plots for morph occurrence
generate_pod_occurrence_plot <- function(
  data, 
  variable, 
  sample_size_data, 
  color, 
  axis_title_x = TRUE, 
  axis_text_x = TRUE, 
  axis_text_y = TRUE,
  axis_title_y = TRUE,
  plot_title = NULL,
  axis_title_size = 10, 
  axis_text_size = 8
) {
  data %>%
    ggplot(aes(x = Microhabitat, fill = factor(!!sym(variable)))) +
    geom_bar(position = "fill") +
    scale_fill_manual(
      values = c("1" = color, "0" = "lightgrey"),
      labels = c("1" = "Present", "0" = "Absent")
    ) +
    labs(
      title = plot_title,
      x = if (isTRUE(axis_title_x)) "Microhabitat" else NULL,
      y = if (isTRUE(axis_title_y)) "Proportion of Quadrats" else NULL,
      fill = NULL
    ) +
    theme_bw(base_size = 8) +
    theme(
      axis.title.y = if (axis_title_y) element_text(size = axis_title_size) else element_blank(),
      axis.text.y  = if (axis_text_y) element_text(size = axis_text_size) else element_blank(),
      axis.title.x = if (axis_title_x) element_text(size = axis_title_size) else element_blank(),
      axis.text.x  = if (axis_text_x) element_text(size = axis_text_size, angle = 45, hjust = 1) else element_blank(),
      legend.position = "none",
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank()
    ) +
    geom_text(
      data = sample_size_data,
      aes(x = Microhabitat, y = 1.05, label = paste("n =", sample_size)),
      vjust = 0, size = 2, inherit.aes = FALSE
    )
}

# Function to create a grey bar with a label
create_top_bar <- function(label) {
    grid::gTree(
        children = grid::gList(
            grid::rectGrob(gp = grid::gpar(fill = "grey90", col = NA), height = unit(2, "lines")),
            grid::textGrob(label, gp = grid::gpar(fontsize = 12), y = 0.5)
        )
    )
}

# Add grey bars with labels to plots
add_grey_bar <- function(plot, label) {
    wrap_elements(create_top_bar(label)) / plot +
        plot_layout(heights = c(0.2, 1)) # Adjust heights for better spacing
}

# Data preparation for Model 4 (Morph Occurrence)
data_m3_sample_size <- data_m3_clean %>%
    group_by(Microhabitat) %>%
    summarise(sample_size = n(), .groups = "drop")

# Generate Morph plots
p_Red_Beige <- generate_pod_occurrence_plot(
    data = data_m3_clean,
    variable = "Red_Beige",
    sample_size_data = data_m3_sample_size,
    color = "indianred4",
    axis_title_x = FALSE,
    axis_text_x = TRUE,
    axis_title_y = TRUE,
    axis_title_size = 10,
    axis_text_size = 8
)

p_White <- generate_pod_occurrence_plot(
    data = data_m3_clean,
    variable = "White",
    sample_size_data = data_m3_sample_size,
    color = "lightblue4",
    axis_title_x = FALSE,
    axis_text_x = TRUE,
    axis_text_y = FALSE,
    axis_title_y = FALSE,
    axis_title_size = 10,
    axis_text_size = 8
)

p_Bright <- generate_pod_occurrence_plot(
    data = data_m3_clean,
    variable = "Bright",
    sample_size_data = data_m3_sample_size,
    color = "darkorange",
    axis_title_x = FALSE,
    axis_text_x = TRUE,
    axis_text_y = FALSE,
    axis_title_y = FALSE,
    axis_title_size = 10,
    axis_text_size = 8
)

# Add grey bars for Model 4
p_Red_Beige_with_bar <- add_grey_bar(p_Red_Beige, "Red_Beige")
p_White_with_bar <- add_grey_bar(p_White, "White")
p_Bright_with_bar <- add_grey_bar(p_Bright, "Bright")

# Combine plots for Model 4
plot_m3_stack <- (p_Red_Beige_with_bar | p_White_with_bar | p_Bright_with_bar) +
    plot_layout(guides = "collect") & theme(plot.margin = margin(3, 0, 3, 0))


# Render the combined plot into a grob
plot_m3_stack_grob <- cowplot::as_grob(plot_m3_stack)


plot_m3_occurrence <- grid.arrange(
    plot_m3_stack_grob,
    ncol = 1,
    heights = unit(c(0.95, 0.05), "npc") # Adjust heights for plot and label
)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_m3_occurrence.png", plot = plot_m3_occurrence, width = 3, height = 3, units = "in", dpi = 300)


In [ ]:
# Convert images to base64
plot_m3_occurrence <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_m3_occurrence.png")

# Create the HTML 
html_occurrence <- paste0("
  <div style='text-align: center; margin: 20px auto; max-width: 600px;'>
    <img src='", plot_m3_occurrence, "' alt='Podocerus Occurrence Plot' style='width: 100%; height: auto; border: 1px solid #ccc;'/>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_occurrence)


### 4. Justification for GLMs and Bayesian methods

#### **Why GLMs?**

Generalized Linear Models (GLMs) are used because our response variable is binary. Linear models assume a continuous range of values for the response variable, which is not suitable for binary outcomes. GLMs address this limitation by introducing a link function that transforms the linear predictor into a form suitable for the response variable's distribution.

---

#### **Why Bayesian methods?**

Bayesian GLMs were chosen over frequentist approaches because:

-   **Sparse data**: Bayesian methods handle sparse datasets more robustly.

-   **Priors**: They allow us to incorporate prior knowledge, improving model performance.

-   **Posterior distributions**: Bayesian models provide posterior distributions, offering a full view of parameter uncertainty.

----

### 5. Define priors

#### **What are priors?**

In Bayesian statistics, priors represent our beliefs about parameter values before analyzing the data. These beliefs are mathematically expressed as probability distributions. Priors guide the model, especially when data are sparse or when the signal-to-noise ratio is low.

---

#### **Why priors matter:**

-   **Prevent overfitting**: Priors discourage extreme parameter estimates unless strongly supported by the data.

-   **Balance restrictiveness and flexibility**: Weakly informative priors let the data dominate while providing reasonable bounds.

-   **Leverage existing knowledge**: Informative priors incorporate previous research or domain expertise, improving accuracy in well-studied systems.

---

#### **How priors work in this analysis:**
We combine the priors (representing initial beliefs) with the likelihood of the observed data to compute posterior distributions, which reflect updated beliefs after observing the data.

The general formula is:

$$
\begin{aligned}
\text{Posterior } \alpha \text{ Likelihood} * \text{Prior}
\end{aligned}
$$

---

#### **Model family and formula**

The response variable (Pod presence) is binary (1 = present, 0 = absent). We model this variable using a Bernoulli distribution with a logit link function:
$$
\begin{aligned}
& y_{i} \sim \text{Bernoulli}(p_{i}) \\
\end{aligned}
$$

The probability of occurrence $p_i$ is modeled as:

$$
\text{log-Odds} = \text{logit}(p_{i}) = \beta_{0} + \beta x_{i} \\
$$

Where:

-   **$y_{i}$**: Response variable for observation $i$

-   **$p_{i}$**: Probability of *P. cristatus* presence for $i$ (on the log scale)

-   **$\beta_{0}$**: Intercept, mean value of $\text{logit}(p)$ (i.e., log-Odds) when predictors are at baseline

-   **$\beta$**: Coefficient/slope for predictor $x_{i}$, representing the effect of $x_{i}$ on $\text{logit}(p_i)$ (i.e., log-Odds).

---

#### Chosen priors and rationale

**Intercept (\beta_{0})**: The intercept (\beta_{0}) reflects the baseline log-odds of *P. cristatus* occurrence in the reference conditions (Red Algae with no nudibranchs). Preliminary SCUBA observations of quadrats containing *P. cristatus* suggest a baseline probability of approximately 75% for Red-Beige morph presence, 20% for White morphs presence, and 5% for Bright morphs presence, translating to:

$$
\begin{aligned}
\text{Red-Beige: } \text{logit}(0.75) = log(\frac{0.75}{1-0.75}) = 1.10 \\
\text{White: } \text{logit}(0.20) = log(\frac{0.20}{1-0.20}) = -1.39 \\
\text{Bright: } \text{logit}(0.05) = log(\frac{0.05}{1-0.05}) = -2.94
\end{aligned}
$$

We set a weakly informative prior for each:
$$
\begin{aligned}
\text{Red-Beige: } \beta_{0} \approx N(1.10,1) \\
\text{White: } \beta_{0} \approx N(-1.39,1) \\
\text{Bright: } \beta_{0} \approx N(-2.94,1)
\end{aligned}
$$

These priors allow for moderate variability in log-odds (95% of values between -2 and 2), corresponding to probabilities ranging from 29% to 96% (Red-Beige), 3% to 65% (White), and 0.7% to 28% (Bright) on the original scale. For example (using Red-Beige):

$$
\begin{aligned}
p = \frac{e^{\text{logit}(p)}}{1 + e^{\text{logit}(p)}}
\end{aligned}
$$

**Red-Beige:**

For $\text{logit}(p)=1.10$:
$$
\begin{aligned}
p = \frac{e^{1.10}}{1 + e^{1.10}} = \frac{3}{1+3} = 0.75
\end{aligned}
$$

For $\text{logit}(p)=-0.9$:
$$
\begin{aligned}
p = \frac{e^{-0.9}}{1 + e^{-0.9}} = \frac{0.41}{1+0.41} = 0.29
\end{aligned}
$$

For $\text{logit}(p)=3.10$:
$$
\begin{aligned}
p = \frac{e^{3.10}}{1 + e^{3.10}} = \frac{22.2}{1+22.2} = 0.96
\end{aligned}
$$

**White:**

For $\text{logit}(p)=-1.39$:
$$
\begin{aligned}
p = \frac{e^{-1.39}}{1 + e^{-1.39}} = \frac{0.249}{1+0.249} = 0.75
\end{aligned}
$$

For $\text{logit}(p)=-3.39$:
$$
\begin{aligned}
p = \frac{e^{-3.39}}{1 + e^{-3.39}} = \frac{0.034}{1+0.034} = 0.03
\end{aligned}
$$

For $\text{logit}(p)=0.61$:
$$
\begin{aligned}
p = \frac{e^{0.61}}{1 + e^{0.61}} = \frac{1.84}{1+1.84} = 0.65
\end{aligned}
$$

**Bright:**

For $\text{logit}(p)=-2.94$:
$$
\begin{aligned}
p = \frac{e^{-2.94}}{1 + e^{-2.94}} = \frac{0.053}{1+0.053} = 0.05
\end{aligned}
$$

For $\text{logit}(p)=-4.94$:
$$
\begin{aligned}
p = \frac{e^{-4.94}}{1 + e^{-4.94}} = \frac{0.007}{1+0.007} = 0.007
\end{aligned}
$$

For $\text{logit}(p)=-0.94$:
$$
\begin{aligned}
p = \frac{e^{-0.94}}{1 + e^{-0.94}} = \frac{0.39}{1+0.39} = 0.28
\end{aligned}
$$

---

**Slope ($\beta$)**: The slope represents the change in log-odds of occurrence for a one-unit increase in the predictor. On the odds scale, the slope is interpreted as a multiplicative scaling factor:

$$
\begin{aligned}
\text{Odds}_{x+1} = \text{Odds}_{x} * e^{\beta}
\end{aligned}
$$

For example:

-   If $\beta = 0.1$, a one-unit increase in the predictor scales the Odds response by $e^{0.1} \approx 1.10$ (a 10% increase).

-   If $\beta = -0.1$, a one-unit increase scales the Odds response by $e^{-0.1} \approx 0.91$ (a 9% decrease).

We don't know how our predictors will affect the response, so we use a weakly informative prior:

$$
\begin{aligned}
\beta \approx N(0,0.5)
\end{aligned}
$$

This prior assumes no effect of the predictor on average ($e^{0} = 1$, and allows moderate positive or negative effects, spanning approximately $[-1, 1]$ on the log scale ($e^{-1} \approx 0.37$ to $e^{1} \approx 2.72$ on the original scale). So the Odds of occurrence will not exceed a minimum of $\text{Odds} * 0.37$ and maximum of $\text{Odds} * 2.72$.

---

**Random effects ($\sigma$)**: Random effects account for variability between groups that we are not explicitly testing. In this model, sampling site will be the random effect. This is because we have repeated measures within the same Sites. Measurements within the same Site are likely to be correlated due to shared environmental factors, spatial proximity, or other unobserved factors specific to the site. Including random effects in our model accounts for this. To capture moderate variability, we assign:

$$
\begin{aligned}
\approx N(0,0.5)
\end{aligned}
$$

This ensures flexibility without overfitting.



---

#### **Visualizing priors**

To validate these priors, we run models sampling only from the priors (sample_prior = "only") and inspect their outputs to ensure they align with our expectations.


>**Note:** We will run these models in RStudio to be consistent because the rstan package sometimes does not like Jupyter. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# m3_Red_Beige_priors <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Red_Beige ~ 1 + Microhabitat + Nudibranch + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(1.10, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = "only",
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3_Red_Beige_priors, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3_Red_Beige_priors.rds")


In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# m3_White_priors <- brm(
#     family = bernoulli(link = "logit"),
#     formula = White ~ 1 + Microhabitat + Nudibranch + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(-1.39, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = "only",
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3_White_priors, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3_White_priors.rds")


In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# m3_Bright_priors <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Bright ~ 1 + Microhabitat + Nudibranch + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(-2.94, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = "only",
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3_Bright_priors, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3_Bright_priors.rds")

In [ ]:
# Load models
m3_Red_Beige_priors <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3_Red_Beige_priors.rds")
m3_White_priors <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3_White_priors.rds")
m3_Bright_priors <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3_Bright_priors.rds")

# Extract results.
prior_samples_m3_Red_Beige <- as_draws_df(m3_Red_Beige_priors)
prior_samples_m3_Red_Beige <- as.data.frame(prior_samples_m3_Red_Beige)

prior_samples_m3_White <- as_draws_df(m3_White_priors)
prior_samples_m3_White <- as.data.frame(prior_samples_m3_White)

prior_samples_m3_Bright <- as_draws_df(m3_Bright_priors)
prior_samples_m3_Bright <- as.data.frame(prior_samples_m3_Bright)

In [ ]:
# List of datasets and labels
prior_samples <- list(
    Red_Beige = prior_samples_m3_Red_Beige,
    White = prior_samples_m3_White,
    Bright = prior_samples_m3_Bright
)


#Custom labels
custom_labels_priors <- c(
    "b_MicrohabitatHydroid_Bryozoa" = "Hydroids with Bryozoans",
    "b_MicrohabitatRed_Algae" = "Red Algae",
    "b_Nudibranch1" = "Nudibranch"
)

custom_labels_priors_intercept <- c(
  "b_Intercept" = "Intercept"
)


# Generate plots for predictors
prior_plots <- lapply(
    names(prior_samples),
    function(label) {
        generate_posterior_plot(
            prior_samples[[label]],
            regex_pars = c(
                "b_MicrohabitatHydroid_Bryozoa",
                "b_MicrohabitatRed_Algae",
                "b_Nudibranch1"
            ),
            x_range = c(-5, 5),
            custom_labels = custom_labels_priors,
            axis_title_y = label %in% c("Red_Beige")
        )
    }
)
names(prior_plots) <- names(prior_samples)


# Generate plot for intercept
prior_plots_intercept <- lapply(
    names(prior_samples),
    function(label) {
        generate_posterior_plot(
            prior_samples[[label]],
            regex_pars = c(
                "b_Intercept"
            ),
            x_range = c(-10, 10),
            custom_labels = custom_labels_priors_intercept,
            axis_title_y = label %in% c("Red_Beige")
        )
    }
)
names(prior_plots_intercept) <- names(prior_samples)



# Add grey bars with labels for each plot
prior_plots_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Adjust heights to reduce spacing
    },
    prior_plots,
    names(prior_plots),
    SIMPLIFY = FALSE
)

prior_plots_intercept_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Adjust heights to reduce spacing
    },
    prior_plots_intercept,
    names(prior_plots_intercept),
    SIMPLIFY = FALSE
)



# Combine plots into a 2x3 grid
plot_priors_m3 <- patchwork::wrap_plots(
    prior_plots_with_bars[c("Red_Beige", "White", "Bright")],
    ncol = 3
) +
    patchwork::plot_layout(guides = "collect")

plot_priors_intercept_m3 <- patchwork::wrap_plots(
    prior_plots_intercept_with_bars[c("Red_Beige", "White", "Bright")],
    ncol = 3
) +
    patchwork::plot_layout(guides = "collect")




# Add a unified x-axis label
plot_priors_m3 <- plot_priors_m3 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 10))
    )

plot_priors_intercept_m3 <- plot_priors_intercept_m3 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.5, size = 10))
    )


ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_m3.png", plot = plot_priors_m3, width = 4.5, height = 4.5, units = "in", dpi = 300)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_intercept_m3.png", plot = plot_priors_intercept_m3, width = 4.5, height = 4.5, units = "in", dpi = 300)


In [ ]:
# Convert images to base64 (assuming these return base64 data URIs)
plot_priors_m3 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_m3.png")
plot_priors_intercept_m3 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_intercept_m3.png")

# Create the HTML 
html_priors <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", plot_priors_m3, "' alt='Posterior Plot'>
    <img src='", plot_priors_intercept_m3, "' alt='Interaction Plot'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_priors)



---

### 6. Run final models

Now that we have finalized the model parameters, we fit models using the actual data and compare them to null models to assess the significance of predictors.


>**Note:** We will run these models in RStudio to be consistent because the rstan package sometimes does not like Jupyter. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# # null model (for comparison)
# m3v0_Red_Beige <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Red_Beige ~ 1 + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(1.10, 1), class = "Intercept"),
#         prior(normal(0, 1), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3v0_Red_Beige, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_Red_Beige.rds")


# # Set seed for reproducibility
# set.seed(5678)
# # Red-Beige Model
# m3v1_Red_Beige <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Red_Beige ~ 1 + Microhabitat + Nudibranch + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(1.10, 1), class = "Intercept"),
#         prior(normal(0, 1), class = "b"),
#         prior(normal(0, 1), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3v1_Red_Beige, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_Red_Beige.rds")

In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# # null model (for comparison)
# m3v0_White <- brm(
#     family = bernoulli(link = "logit"),
#     formula = White ~ 1 + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(-1.39, 1), class = "Intercept"),
#         prior(normal(0, 1), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3v0_White, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_White.rds")


# # Set seed for reproducibility
# set.seed(5678)
# # White Model
# m3v1_White <- brm(
#     family = bernoulli(link = "logit"),
#     formula = White ~ 1 + Microhabitat + Nudibranch + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(-1.39, 1), class = "Intercept"),
#         prior(normal(0, 1), class = "b"),
#         prior(normal(0, 1), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3v1_White, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_White.rds")


In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# # null model (for comparison)
# m3v0_Bright <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Bright ~ 1 + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(-2.94, 1), class = "Intercept"),
#         prior(normal(0, 1), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3v0_Bright, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_Bright.rds")


# # Set seed for reproducibility
# set.seed(5678)
# #Bright Model
# m3v1_Bright <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Bright ~ 1 + Microhabitat + Nudibranch + (1 | Site),
#     data = data_m3_clean,
#     prior = c(
#         prior(normal(-2.94, 1), class = "Intercept"),
#         prior(normal(0, 1), class = "b"),
#         prior(normal(0, 1), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m3v1_Bright, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_Bright.rds")



#### **Model comparison**

Make sure final model outperforms the null model. Use LOO to compare models. We are examining whether nudibranch occurrence influences P. cristatus distributions overall while controlling for microhabitat associations, so both nudibranchs and microhabitat are important to our question, so each model should have both predictors (i.e., there will be no single predictor models). Also, we will omit nudibranch-microhabitat interactions since it was shown in Model 3 that this interaction is only important for responses at the Lafoea level.


>**Note:** We will added loo to these models in RStudio. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:
# # Add LOO to models
# m3v0_Red_Beige <- add_criterion(m3v0_Red_Beige, "loo", moment_match = TRUE)
# m3v0_White <- add_criterion(m3v0_White, "loo", moment_match = TRUE)
# m3v0_Bright <- add_criterion(m3v0_Bright, "loo", moment_match = TRUE)
# m3v1_Red_Beige <- add_criterion(m3v1_Red_Beige, "loo", moment_match = TRUE)
# m3v1_White <- add_criterion(m3v1_White, "loo", moment_match = TRUE)
# m3v1_Bright <- add_criterion(m3v1_Bright, "loo", moment_match = TRUE)

# Save models with loo so you don't have to do this again
# saveRDS(m3v0_Red_Beige, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_Red_Beige.rds")
# saveRDS(m3v0_White, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_White.rds")
# saveRDS(m3v0_Bright, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_Bright.rds")
# saveRDS(m3v1_Red_Beige, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_Red_Beige.rds")
# saveRDS(m3v1_White, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_White.rds")
# saveRDS(m3v1_Bright, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_Bright.rds")

In [ ]:
# Load models from R
m3v0_Red_Beige <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_Red_Beige.rds")
m3v0_White <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_White.rds")
m3v0_Bright <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v0_Bright.rds")
m3v1_Red_Beige <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_Red_Beige.rds")
m3v1_White <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_White.rds")
m3v1_Bright <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m3v1_Bright.rds")


In [ ]:

# Compare models with loo
loo1 <- loo_compare(m3v0_Red_Beige, m3v1_Red_Beige)
loo2 <- loo_compare(m3v0_White, m3v1_White)
loo3 <- loo_compare(m3v0_Bright, m3v1_Bright)

In [ ]:
# Convert to dataframe
df_loo1 <- as.data.frame(loo1) %>%
  rownames_to_column(var = "Model")

df_loo2 <- as.data.frame(loo2) %>%
  rownames_to_column(var = "Model")

df_loo3 <- as.data.frame(loo3) %>%
  rownames_to_column(var = "Model")


# Save loo as tables
write.table(df_loo1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_loo_m3_Red_Beige.csv", sep = ",", row.names = TRUE, col.names = TRUE)

write.table(df_loo2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_loo_m3_White.csv", sep = ",", row.names = TRUE, col.names = TRUE)

write.table(df_loo2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_loo_m3_Bright.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Convert each data frame to a plain HTML table string
table_1_loo <- minimal_html_table(df_loo1, caption = "LOO values - Probability of Red-Beige Morph Occurrence")
table_2_loo <- minimal_html_table(df_loo2, caption = "LOO values - Probability of White Morph Occurrence")
table_3_loo <- minimal_html_table(df_loo3, caption = "LOO values - Probability of Bright Morph Occurrence")

my_tabs_loo <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_loo:checked ~ #content1_loo,
#tab2_loo:checked ~ #content2_loo,
#tab3_loo:checked ~ #content3_loo {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_loo:checked + label[for="tab1_loo"],
#tab2_loo:checked + label[for="tab2_loo"],
#tab3_loo:checked + label[for="tab3_loo"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_loo" id="tab1_loo" checked>
  <label class="tab-label" for="tab1_loo">Table 1</label>

  <!-- 2) Tab radio + label -->
  <input type="radio" name="tabs_loo" id="tab2_loo">
  <label class="tab-label" for="tab2_loo">Table 2</label>

  <!-- 3) Tab radio + label -->
  <input type="radio" name="tabs_loo" id="tab3_loo">
  <label class="tab-label" for="tab3_loo">Table 3</label>


  <!-- Content for each tab -->
  <div class="tab-content" id="content1_loo">REPLACE_WITH_table_1</div>
  <div class="tab-content" id="content2_loo">REPLACE_WITH_table_2</div>
  <div class="tab-content" id="content3_loo">REPLACE_WITH_table_3</div>
</div>
'

# Now do the replacements for each table
my_tabs_loo <- gsub("REPLACE_WITH_table_1", table_1_loo, my_tabs_loo)
my_tabs_loo <- gsub("REPLACE_WITH_table_2", table_2_loo, my_tabs_loo)
my_tabs_loo <- gsub("REPLACE_WITH_table_3", table_3_loo, my_tabs_loo)

IRdisplay::display_html(my_tabs_loo)


#### **Visualize posteriors**

We extract the final model results and visualize the posteriors of our model parameters to get an idea of the significance of the results. This is what we did above when we were evaluating our priors.

In [ ]:

posterior_samples_m3v1_Red_Beige <- as_draws_df(m3v1_Red_Beige)
posterior_samples_m3v1_Red_Beige <- as.data.frame(posterior_samples_m3v1_Red_Beige)

posterior_samples_m3v1_White <- as_draws_df(m3v1_White)
posterior_samples_m3v1_White <- as.data.frame(posterior_samples_m3v1_White)

posterior_samples_m3v1_Bright <- as_draws_df(m3v1_Bright)
posterior_samples_m3v1_Bright <- as.data.frame(posterior_samples_m3v1_Bright)


In [ ]:

# List of datasets and labels
posterior_samples <- list(
    Red_Beige = posterior_samples_m3v1_Red_Beige,
    White = posterior_samples_m3v1_White,
    Bright = posterior_samples_m3v1_Bright
)


# Baseline category data
baseline_data <- tibble(
  parameter = "Hydroid",
  mean = 0,  # Centered at 0
  ci_low = -0.2,  # Example CI range
  ci_high = 0.2
)

# Order categories so "Red Algae" appears at the bottom
parameter_order <- c(
  "Hydroid",
  "b_MicrohabitatHydroid_Bryozoa",
  "b_MicrohabitatRed_Algae",
  "b_Nudibranch1"
)

#Custom labels
custom_labels_posteriors <- c(
  "Hydroid" = "Hydroids",
  "b_MicrohabitatHydroid_Bryozoa" = "Hydroids with Bryozoans",
  "b_MicrohabitatRed_Algae" = "Red Algae",
  "b_Nudibranch1" = "Aeolid Nudibranch Presence"
)


# Convert the y-axis parameters to factors for consistent alignment
baseline_data$parameter <- factor(baseline_data$parameter, levels = parameter_order)


# Generate plots for predictors
posterior_plots <- lapply(
    names(posterior_samples),
    function(label) {
        generate_posterior_plot(
            posterior_samples[[label]],
            regex_pars = c(
                "b_MicrohabitatHydroid_Bryozoa",
                "b_MicrohabitatRed_Algae",
                "b_Nudibranch1"
            ),
            x_range = c(-3, 3),
            custom_labels = custom_labels_posteriors,
            axis_title_y = label %in% c("Red_Beige")
        ) +
        geom_point(
            data = baseline_data,
            aes(x = mean, y = parameter),
            inherit.aes = FALSE,
            color = "dodgerblue4",
            size = 2
        )
    }
)
names(posterior_plots) <- names(posterior_samples)



# Add grey bars with labels for each plot
top_bar_labels <- c("Red-Beige", "White", "Bright")

posterior_plots_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Adjust heights to reduce spacing
    },
    posterior_plots,
    top_bar_labels,
    SIMPLIFY = FALSE
)


# Combine plots into a 2x3 grid
plot_posteriors_m3v1 <- patchwork::wrap_plots(
    posterior_plots_with_bars[c("Red_Beige", "White", "Bright")],
    ncol = 3,
) +
    patchwork::plot_layout(guides = "collect")


# Add a unified x-axis label
plot_posteriors_m3v1 <- plot_posteriors_m3v1 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 16))
    ) + 
  theme(
    axis.title.y = element_text(size = 16),
    axis.text.x  = element_text(size = 10),
    axis.text.y  = element_text(size = 10)
  )

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m3v1.png", plot = plot_posteriors_m3v1, width = 4.5, height = 4.5, units = "in", dpi = 300)


In [ ]:
# Convert images to base64
plot_posteriors_m3v1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m3v1.png")

# Create the HTML 
html_posteriors <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", plot_posteriors_m3v1, "' alt='Posterior Plot'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_posteriors)



The graph of the posteriors gives us an idea of the significance of each predictor. We need to follow up with an evaluation of the model performance before we can trust these results.

---

### 7. Evaluate model performance

#### **Trace plots**

Visualize parameter sampling across iterations to confirm convergence. Each chain should wander around the same mean value without any strong upward or downward trends. "fuzzy caterpillar" or "horizontal band." 

In [ ]:
trace_plots_m3v1_Red_Beige <- generate_trace_plot(
  model = m3v1_Red_Beige,
  regex_pars = c("b_MicrohabitatHydroid_Bryozoa", "b_MicrohabitatRed_Algae", "b_Nudibranch1"),
  plot_title = "Red-Beige Occurrence Model",
  axis_title_y = TRUE,
  axis_text_x = TRUE,
  axis_title_size = 10,
  axis_text_size = 8
)

trace_plots_m3v1_White <- generate_trace_plot(
  model = m3v1_White,
  regex_pars = c("b_MicrohabitatHydroid_Bryozoa", "b_MicrohabitatRed_Algae", "b_Nudibranch1"),
  plot_title = "White Occurrence Model",
  axis_title_y = TRUE,
  axis_text_x = TRUE,
  axis_title_size = 10,
  axis_text_size = 8
)

trace_plots_m3v1_Bright <- generate_trace_plot(
  model = m3v1_Bright,
  regex_pars = c("b_MicrohabitatHydroid_Bryozoa", "b_MicrohabitatRed_Algae", "b_Nudibranch1"),
  plot_title = "Bright Occurrence Model",
  axis_title_y = TRUE,
  axis_text_x = TRUE,
  axis_title_size = 10,
  axis_text_size = 8
)


ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m3v1_Red_Beige.png", plot = trace_plots_m3v1_Red_Beige, width = 6, height = 3, units = "in", dpi = 300)
ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m3v1_White.png", plot = trace_plots_m3v1_White, width = 6, height = 3, units = "in", dpi = 300)
ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m3v1_Bright.png", plot = trace_plots_m3v1_Bright, width = 6, height = 3, units = "in", dpi = 300)


In [ ]:
# Convert images to base64 (if not already done)
trace_plots_m3v1_Red_Beige <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m3v1_Red_Beige.png")
trace_plots_m3v1_White     <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m3v1_White.png")
trace_plots_m3v1_Bright    <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m3v1_Bright.png")


# Create the HTML 
html_trace_plots <- paste0("
<style>
  .grid-container {
    display: grid;
    grid-template-columns: repeat(1, 1fr); /* 1 columns per row */
    gap: 3px;
    padding: 3px;
    justify-items: center;
  }

  .grid-container img {
    max-width: 600px;
    width: 100%;
    height: auto;
    border: 1px solid #ccc;
  }
</style>

<div class='grid-container'>
  <img src='", trace_plots_m3v1_Red_Beige, "' alt='Red-Beige Trace Plot'>
  <img src='", trace_plots_m3v1_White, "' alt='White Trace Plot'>
  <img src='", trace_plots_m3v1_Bright, "' alt='Bright Trace Plot'>
</div>
")


# Display the HTML
IRdisplay::display_html(html_trace_plots)





#### **Posterior predictive checks**

Simulate data based on the models and compare to observed data to verify goodness-of-fit. We do this using "pp_check". The observed data (black line/dots) should sit comfortably within the distribution of simulated data (colored areas or lines)

In [ ]:

# Generate posterior predictive check plots for all models
pp_check_plots_m3v1_Red_Beige <-  generate_pp_check(
  model = m3v1_Red_Beige,
  nreps = 100,
  axis_title_y = TRUE,
  y_label = "Density",
  plot_title = "Red-Beige Occurrence Model",
  axis_title_size = 10,  # custom title size
  axis_text_size = 8    # custom tick label size
)

pp_check_plots_m3v1_White <-  generate_pp_check(
  model = m3v1_White,
  nreps = 100,
  axis_title_y = TRUE,
  y_label = "Density",
  plot_title = "White Occurrence Model",
  axis_title_size = 10,  # custom title size
  axis_text_size = 8    # custom tick label size
)

pp_check_plots_m3v1_Bright <-  generate_pp_check(
  model = m3v1_Bright,
  nreps = 100,
  axis_title_y = TRUE,
  y_label = "Density",
  plot_title = "Bright Occurrence Model",
  axis_title_size = 10,  # custom title size
  axis_text_size = 8    # custom tick label size
)


ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m3v1_Red_Beige.png", plot = pp_check_plots_m3v1_Red_Beige, width = 3, height = 3, units = "in", dpi = 300)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m3v1_White.png", plot = pp_check_plots_m3v1_White, width = 3, height = 3, units = "in", dpi = 300)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m3v1_Bright.png", plot = pp_check_plots_m3v1_Bright, width = 3, height = 3, units = "in", dpi = 300)


In [ ]:
# Convert images to base64
pp_check_plots_m3v1_Red_Beige <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m3v1_Red_Beige.png")
pp_check_plots_m3v1_White     <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m3v1_White.png")
pp_check_plots_m3v1_Bright    <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m3v1_Bright.png")


# Create the HTML (horizontal display)
html_pp_check_plots <- paste0("
<style>
  .grid-container {
    display: grid;
    grid-template-columns: repeat(3, 1fr); /* 3 columns per row */
    gap: 15px;
    padding: 10px;
    justify-items: center;
  }

  .grid-container img {
    max-width: 300px;
    width: 100%;
    height: auto;
    border: 1px solid #ccc;
  }
</style>

<div class='grid-container'>
  <img src='", pp_check_plots_m3v1_Red_Beige, "' alt='Red-Beige Plot'>
  <img src='", pp_check_plots_m3v1_White, "' alt='White Plot'>
  <img src='", pp_check_plots_m3v1_Bright, "' alt='Bright Plot'>
</div>
")

IRdisplay::display_html(html_pp_check_plots)


#### **Check convergence**

Check that all $\hat{R}$ values are close to 1, indicating good convergence.

In [ ]:
# Create a helper function
extract_rhat <- function(model, model_name) {
  rhat(model) %>%
    as.data.frame() %>%
    rownames_to_column(var = "Parameter") %>%
    dplyr::filter(startsWith(Parameter, "b_")) %>%   # <-- keep only b_ terms
    dplyr::rename(Rhat = 2) %>%
    dplyr::mutate(Model = model_name) %>%
    dplyr::mutate(across(where(is.numeric), ~ signif(.x, digits = 3)))
}

# Extract for each model group
rhat1 <- extract_rhat(m3v1_Red_Beige, "Red_Beige")
rhat2 <- extract_rhat(m3v1_White,     "White")
rhat3 <- extract_rhat(m3v1_Bright,    "Bright")

In [ ]:
# Save tables
write.table(rhat1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_rhat_m3v1_Red_Beige.csv", sep = ",", row.names = FALSE, col.names = TRUE)
write.table(rhat2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_rhat_m3v1_White.csv", sep = ",", row.names = FALSE, col.names = TRUE)
write.table(rhat3, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_rhat_m3v1_Bright.csv", sep = ",", row.names = FALSE, col.names = TRUE)

In [ ]:
# Convert each data frame to a plain HTML table string
table_1_rhat <- minimal_html_table(rhat1, caption = "Rhat values - Probability of Red-Beige Morph Occurrence")
table_2_rhat <- minimal_html_table(rhat2, caption = "Rhat values - Probability of White Morph Occurrence")
table_3_rhat <- minimal_html_table(rhat3, caption = "Rhat values - Probability of Bright Morph Occurrence")


my_tabs_rhat <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_rhat:checked ~ #content1_rhat,
#tab2_rhat:checked ~ #content2_rhat,
#tab3_rhat:checked ~ #content3_rhat {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_rhat:checked + label[for="tab1_rhat"],
#tab2_rhat:checked + label[for="tab2_rhat"],
#tab3_rhat:checked + label[for="tab3_rhat"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_rhat" id="tab1_rhat" checked>
  <label class="tab-label" for="tab1_rhat">Table 1</label>

  <!-- 2) Tab radio + label -->
  <input type="radio" name="tabs_rhat" id="tab2_rhat">
  <label class="tab-label" for="tab2_rhat">Table 2</label>

  <!-- 3) Tab radio + label -->
  <input type="radio" name="tabs_rhat" id="tab3_rhat">
  <label class="tab-label" for="tab3_rhat">Table 3</label>

  <!-- Content for each tab -->
  <div class="tab-content" id="content1_rhat">REPLACE_WITH_table_1</div>
  <div class="tab-content" id="content2_rhat">REPLACE_WITH_table_2</div>
  <div class="tab-content" id="content3_rhat">REPLACE_WITH_table_3</div>
</div>
'

# Now do the replacements for each table
my_tabs_rhat <- gsub("REPLACE_WITH_table_1", table_1_rhat, my_tabs_rhat)
my_tabs_rhat <- gsub("REPLACE_WITH_table_2", table_2_rhat, my_tabs_rhat)
my_tabs_rhat <- gsub("REPLACE_WITH_table_3", table_3_rhat, my_tabs_rhat)

IRdisplay::display_html(my_tabs_rhat)



#### **Check uncertainty**

Extract parameter estimates and their confidence intervals to assess the significance of the predictors on color pattern metrics. We check 85% and 95% confidence intervals. Summaries are displayed in tables for all models. 

In [ ]:
# Extract summaries for each variable
extract_summary <- function(model, prob_85, prob_95) {
    summary_85 <- summary(model, prob = prob_85)
    summary_95 <- summary(model, prob = prob_95)

    as.data.frame(summary_85$fixed) %>%
        dplyr::select("Estimate", "Est.Error", "l-85% CI", "u-85% CI") %>%
        mutate(
            "l-95% CI" = summary_95$fixed$`l-95% CI`,
            "u-95% CI" = summary_95$fixed$`u-95% CI`
        ) %>%
        mutate(across(where(is.numeric), ~ signif(.x, digits = 3))) %>%
        rownames_to_column(var = "Parameter") # Add rownames as Parameter column
}

uncertainty1 <- extract_summary(m3v1_Red_Beige, prob_85 = 0.85, prob_95 = 0.95)
uncertainty2 <- extract_summary(m3v1_White, prob_85 = 0.85, prob_95 = 0.95)
uncertainty3 <- extract_summary(m3v1_Bright, prob_85 = 0.85, prob_95 = 0.95)


In [ ]:

# Save tables
write.table(uncertainty1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_uncertainty_m3v1_Red_Beige.csv", sep = ",", row.names = TRUE, col.names = TRUE)

write.table(uncertainty2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_uncertainty_m3v1_White.csv", sep = ",", row.names = TRUE, col.names = TRUE)

write.table(uncertainty3, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_uncertainty_m3v1_Bright.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Convert each data frame to a plain HTML table string
table_1_uncertainty <- minimal_html_table(uncertainty1, caption = "Uncertainty values - Probability of Red-Beige Morph Occurrence")
table_2_uncertainty <- minimal_html_table(uncertainty2, caption = "Uncertainty values - Probability of White Morph Occurrence")
table_3_uncertainty <- minimal_html_table(uncertainty3, caption = "Uncertainty values - Probability of Bright Morph Occurrence")

my_tabs_uncertainty <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_uncertainty:checked ~ #content1_uncertainty,
#tab2_uncertainty:checked ~ #content2_uncertainty,
#tab3_uncertainty:checked ~ #content3_uncertainty {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_uncertainty:checked + label[for="tab1_uncertainty"],
#tab2_uncertainty:checked + label[for="tab2_uncertainty"],
#tab3_uncertainty:checked + label[for="tab3_uncertainty"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_uncertainty" id="tab1_uncertainty" checked>
  <label class="tab-label" for="tab1_uncertainty">Table 1</label>

  <!-- 2) Tab radio + label -->
  <input type="radio" name="tabs_uncertainty" id="tab2_uncertainty">
  <label class="tab-label" for="tab2_uncertainty">Table 2</label>

  <!-- 3) Tab radio + label -->
  <input type="radio" name="tabs_uncertainty" id="tab3_uncertainty">
  <label class="tab-label" for="tab3_uncertainty">Table 3</label>

  <!-- Content for each tab -->
  <div class="tab-content" id="content1_uncertainty">REPLACE_WITH_table_1</div>
  <div class="tab-content" id="content2_uncertainty">REPLACE_WITH_table_2</div>
  <div class="tab-content" id="content3_uncertainty">REPLACE_WITH_table_3</div>
</div>
'

# Now do the replacements for each table
my_tabs_uncertainty <- gsub("REPLACE_WITH_table_1", table_1_uncertainty, my_tabs_uncertainty)
my_tabs_uncertainty <- gsub("REPLACE_WITH_table_2", table_2_uncertainty, my_tabs_uncertainty)
my_tabs_uncertainty <- gsub("REPLACE_WITH_table_3", table_3_uncertainty, my_tabs_uncertainty)

IRdisplay::display_html(my_tabs_uncertainty)


---

#### **Check posterior probabilities**

Now we can evaluate our hypotheses using the posterior distributions of the model parameters. Remember, we are concerned with two things:
1. Is there a difference in the probability of finding different morphs between microhabitats? (Microhabitat effect)
2. Is there a difference in the probability of finding different morphs between quadrats with and without aeolid nudibranchs? (Nudibranch effect)

We will calculate the posterior probability of these effects.

In [ ]:
##### **Posterior Probability of Microhabitat Effect**

draws_Red_Beige <- as_draws_df(m3v1_Red_Beige)
draws_White <- as_draws_df(m3v1_White)
draws_Bright <- as_draws_df(m3v1_Bright)

# Posterior probability that Other Hydroids, Sertulariidae, Hydrallmania, Lafoea, or Sertulariidae with Bryozoa are **more** likely to have RB, W, or Bright Morphs than Red Algae (reference microhabitat)
pp_Red_Beige_Red_Algae_positive <- mean(draws_Red_Beige$b_MicrohabitatRed_Algae > 0)
pp_Red_Beige_Hydroid_Bryozoa_positive <- mean(draws_Red_Beige$b_MicrohabitatHydroid_Bryozoa > 0)
pp_White_Red_Algae_positive <- mean(draws_White$b_MicrohabitatRed_Algae > 0)
pp_White_Hydroid_Bryozoa_positive <- mean(draws_White$b_MicrohabitatHydroid_Bryozoa > 0)
pp_Bright_Red_Algae_positive <- mean(draws_Bright$b_MicrohabitatRed_Algae > 0)
pp_Bright_Hydroid_Bryozoa_positive <- mean(draws_Bright$b_MicrohabitatHydroid_Bryozoa > 0)

# Posterior probability that Other Hydroids, Sertulariidae, Hydrallmania, Lafoea, or Sertulariidae with Bryozoa are **less** likely to have RB, W, or Bright Morphs than Red Algae (reference microhabitat)
pp_Red_Beige_Red_Algae_negative <- mean(draws_Red_Beige$b_MicrohabitatRed_Algae < 0)
pp_Red_Beige_Hydroid_Bryozoa_negative <- mean(draws_Red_Beige$b_MicrohabitatHydroid_Bryozoa < 0)
pp_White_Red_Algae_negative <- mean(draws_White$b_MicrohabitatRed_Algae < 0)
pp_White_Hydroid_Bryozoa_negative <- mean(draws_White$b_MicrohabitatHydroid_Bryozoa < 0)
pp_Bright_Red_Algae_negative <- mean(draws_Bright$b_MicrohabitatRed_Algae < 0)
pp_Bright_Hydroid_Bryozoa_negative <- mean(draws_Bright$b_MicrohabitatHydroid_Bryozoa < 0)


# Create a summary table
pp_summary_m3v1_microhabitat <- data.frame(
  Hypothesis = c(
    "P(Red-Beige Morph Hydroids with Bryozoa effect > 0)",
    "P(Red-Beige Morph Red Algae effect > 0)",
    "P(Red-Beige Morph Hydroids with Bryozoa effect < 0)",
    "P(Red-Beige Morph Red Algae effect < 0)",
    "P(White Morph Hydroids with Bryozoa effect > 0)",
    "P(White Morph Red Algae effect > 0)",
    "P(White Morph Hydroids with Bryozoa effect < 0)",
    "P(White Morph Red Algae effect < 0)",
    "P(Bright Morph Hydroids with Bryozoa effect > 0)",
    "P(Bright Morph Red Algae effect > 0)",
    "P(Bright Morph Hydroids with Bryozoa effect < 0)",
    "P(Bright Morph Red Algae effect < 0)"
  ),
  PosteriorProbability = c(
    pp_Red_Beige_Hydroid_Bryozoa_positive,
    pp_Red_Beige_Red_Algae_positive,
    pp_Red_Beige_Hydroid_Bryozoa_negative,
    pp_Red_Beige_Red_Algae_negative,
    pp_White_Hydroid_Bryozoa_positive,
    pp_White_Red_Algae_positive,
    pp_White_Hydroid_Bryozoa_negative,
    pp_White_Red_Algae_negative,
    pp_Bright_Hydroid_Bryozoa_positive,
    pp_Bright_Red_Algae_positive,
    pp_Bright_Hydroid_Bryozoa_negative,
    pp_Bright_Red_Algae_negative
  )
)

# Save table
write.table(pp_summary_m3v1_microhabitat, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m3v1_microhabitat.csv", sep = ",", row.names = TRUE, col.names = TRUE)

ERROR: Error in as_draws_df(m_Q2m4_Red_Beige): could not find function "as_draws_df"


In [ ]:
# Generate HTML table
html_table <- minimal_html_table(pp_summary_m3v1_microhabitat, caption = "Posterior probabilities for Microhabitat Effect")

# Display it as HTML
IRdisplay::display_html(html_table)

In [ ]:
##### **Posterior Probability of Nudibranch Effect**

draws_Red_Beige <- as_draws_df(m3v1_Red_Beige)
draws_White <- as_draws_df(m3v1_White)
draws_Bright <- as_draws_df(m3v1_Bright)

# Posterior probability that quadrats with Aeolid Nudibranchs are **more** likely to have RB, W, or Bright morphs than quadrats without Aeolid Nudibranchs
pp_Red_Beige_nudie_positive <- mean(draws_Red_Beige$b_Nudibranch1 > 0)
pp_White_nudie_positive <- mean(draws_White$b_Nudibranch1 > 0)
pp_Bright_nudie_positive <- mean(draws_Bright$b_Nudibranch1 > 0)

# Posterior probability that quadrats with Aeolid Nudibranchs are **less** likely to have RB, W, or Bright morphs than quadrats without Aeolid Nudibranchs
pp_Red_Beige_nudie_negative <- mean(draws_Red_Beige$b_Nudibranch1 < 0)
pp_White_nudie_negative <- mean(draws_White$b_Nudibranch1 < 0)
pp_Bright_nudie_negative <- mean(draws_Bright$b_Nudibranch1 < 0)


# Create a summary table
pp_summary_m3v1_nudibranch <- data.frame(
  Hypothesis = c(
    "P(Red-Beige Morph Nudibranch effect > 0)",
    "P(Red-Beige Morph Nudibranch effect < 0)",
    "P(White Morph Nudibranch effect > 0)",
    "P(White Morph Nudibranch effect < 0)",
    "P(Bright Morph Nudibranch effect > 0)",
    "P(Bright Morph Nudibranch effect < 0)"
  ),
  PosteriorProbability = c(
    pp_Red_Beige_nudie_positive,
    pp_Red_Beige_nudie_negative,
    pp_White_nudie_positive,
    pp_White_nudie_negative,
    pp_Bright_nudie_positive,
    pp_Bright_nudie_negative
  )
)

# Save table
write.table(pp_summary_m3v1_nudibranch, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m3v1_nudibranch.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Generate HTML table
html_table <- minimal_html_table(pp_summary_m3v1_nudibranch, caption = "Posterior probabilities for Nudibranch Effect")

# Display it as HTML
IRdisplay::display_html(html_table)


---

### 8. Summarize results

Neither Microhabitat type nor aeolid nudibranch occurrence have a significant effect on the log-odds of P. cristatus morph occurrence at 85% and 95% confidence intervals.

#### **Make final figures**

Generate tidy figures for the Results section of our report/paper. We will limit the results to showing only Microhabitat categories and add the intercept category at 0 for comparison.


In [ ]:
##### FINAL PLOT FOR WNAN PAPER

# Define the predictor list
predictors_m3v1 <- list(
  list(
    name = "Microhabitat",
    model = "m2",
    baseline = tibble(parameter = "Hydroids", mean = 0),
    order = c("Hydroids", "b_MicrohabitatHydroid_Bryozoa", "b_MicrohabitatRed_Algae"),
    labels = c(
      "Hydroids" = "Hydroids",
      "b_MicrohabitatRed_Algae" = "Red algae",
      "b_MicrohabitatHydroid_Bryozoa" = "Bryozoa"
    ),
    regex_pars = c(
      "^b_MicrohabitatRed_Algae$",
      "^b_MicrohabitatHydroid_Bryozoa$"
    )
  ),
  list(
    name = "Nudibranch",
    model = "m2",
    baseline = tibble(parameter = "Aeolid Absence", mean = 0),
    order = c("Aeolid Absence", "b_Nudibranch1"),
    labels = c(
      "Aeolid Absence" = "Aeolid absent",
      "b_Nudibranch1" = "Aeolid present"
    ),
    regex_pars = c("^b_Nudibranch1$")
  )
)

# Posterior samples by morph group
posterior_samples_all <- list(
  RB = posterior_samples_m3v1_Red_Beige,
  W = posterior_samples_m3v1_White,
  Bright = posterior_samples_m3v1_Bright
)

# Extraction function (same syntax)
extract_effects <- function(posterior_df, predictor_cfg, morph_label) {
  matched_cols <- posterior_df %>%
    dplyr::select(matches(paste(predictor_cfg$regex_pars, collapse = "|"))) %>%
    colnames()

  if (length(matched_cols) == 0) {
    stop(paste("No columns matched for predictor:", predictor_cfg$name))
  }

  draws <- posterior_df %>%
    dplyr::select(all_of(matched_cols)) %>%
    pivot_longer(cols = everything(), names_to = "parameter", values_to = "value") %>%
    mutate(
      label = predictor_cfg$labels[parameter],
      predictor = predictor_cfg$name,
      group = predictor_cfg$name,   # ✅ Use predictor name for color fill
      morph = morph_label               # ✅ Use morph for faceting
    )

  if (!is.null(predictor_cfg$baseline)) {
    baseline <- predictor_cfg$baseline %>%
      mutate(
        label = predictor_cfg$labels[parameter],
        predictor = predictor_cfg$name,
        group = predictor_cfg$name,  
        morph = morph_label,
        value = mean
      )
    out <- bind_rows(draws, baseline)
  } else {
    out <- draws
  }

  return(out)
}


# Build combined plot data
plot_data_m3v1 <- map_dfr(
  names(posterior_samples_all),
  function(morph) {
    map_dfr(
      predictors_m3v1,
      function(predictor_cfg) {
        extract_effects(
          posterior_df = posterior_samples_all[[morph]],
          predictor_cfg = predictor_cfg,
          morph_label = morph
        )
      }
    )
  }
)

# Relabel morphs for display
plot_data_m3v1$morph <- factor(
  plot_data_m3v1$morph,
  levels = c("RB", "W", "Bright"),
  labels = c("Red-Beige", "White", "Bright")
)

# Factor label order
label_order_m3v1 <- c(
  "Hydroids", "Red algae", "Bryozoa",
  "Aeolid absent", "Aeolid present"
)
plot_data_m3v1$label <- factor(plot_data_m3v1$label, levels =label_order_m3v1)

# Final plot (facet by group)
final_plot_posteriors_m3v1 <- ggplot(plot_data_m3v1, aes(x = value, y = label, fill = group, color = group)) +
  stat_halfeye(
    .width = c(0.85, 0.95),
    slab_alpha = 0.4,
    interval_size_range = c(0.75, 0),
    normalize = "groups",
    slab_linewidth = 0.6,
    point_size = 1.25
  ) +
  geom_point(
    data = filter(plot_data_m3v1, !is.na(value) & !is.na(mean)),
    aes(x = mean),  # ✅ Match point color to group
    inherit.aes = TRUE,
    size = 0.5,
    shape = 21
  ) +
  geom_vline(xintercept = 0, size = 0.6, linetype = 3, color = "red") +
  facet_wrap(~ morph, ncol = 3, labeller = label_value) +
  scale_fill_manual(
    values = c(
      "Microhabitat" = "forestgreen",
      "Nudibranch" = "red"
    )
  ) +
  scale_color_manual(  # ✅ Matching point colors
    values = c(
      "Microhabitat" = "forestgreen",
      "Nudibranch" = "red"
    )
  ) +
  labs(
    x = "Effect size (logit scale)",
    y = NULL
  ) +
  theme_bw(base_size = 8) +
  theme(
    strip.text = element_text(face = "bold", size = 10),
    axis.text.y = element_text(size = 8),
    panel.spacing = unit(0.5, "lines"),
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    legend.position = "none"
  )


# Save
ggsave(
  "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/final_plot_posteriors_m3v1.png",
  plot = final_plot_posteriors_m3v1,
  width = 8.5, height = 4.5, units = "in", dpi = 300
)


In [ ]:
# Convert images to base64
final_plot_posteriors_m3v1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/final_plot_posteriors_m3v1.png")

# Create the HTML 
html_posteriors_final <- paste0("
  <style>
    .image-row {
      display: flex;
      gap: 20px;
      justify-content: center;
      align-items: flex-start;
    }
    .image-row img {
      max-width: 100%;
      height: auto;
      border: 1px solid #ccc;
    }
  </style>
<div class='image-row'>
  <img src='", final_plot_posteriors_m3v1, "' alt='Final Posterior Plot'>
</div>
")


# Display the HTML
IRdisplay::display_html(html_posteriors_final)